<a href="https://colab.research.google.com/github/POmegrenadier9090/keyboard_arr_optimizer/blob/main/Keyboard_Optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [219]:
from google.colab import drive

drive.mount('/content/drive')
%cd "/content/drive/My Drive/Project"
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Project
gooLab_hiragana.txt  Keyboard_Optimizer.ipynb  sample_text_営業意思決定支援.txt


In [224]:
from sys import setswitchinterval
import numpy as np
import pandas as pd
import math
import requests
import chardet

pd.set_option('display.max_columns', None)

alphabet_index = {
    'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6,
    'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12,
    'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18,
    't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25
}

index_alphabet = {
  0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g',
  7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm',
  13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's',
  19: 't', 20: 'u', 21: 'v', 22: 'w', 23: 'x', 24: 'y', 25: 'z'
}

key_cost = np.array(
        [
            [22,18,10,9,15,15,9,10,18,22],
            [11,7, 1, 1,7, 7, 1,1, 7, 11],
            [22,18,10,9,15,15,9,10,18,22]
    ]
        )



#calculates the coordinates difining "z" as the origin
key_position = np.array(
    [#abcdefghijklmf
     [0,1],[4,0],[2,0],[2,1],[2,2],[3,1],[4,1],[5,1],[7,2],[6,1],[7,1],[8,1],[6,0],
     #nopqrstuvwxyz
     [5,0],[8,2],[9,2],[0,2],[3,2],[1,1],[4,2],[6,2],[3,0],[1,2],[1,0],[5,2],[0,0]
     ]
)
print(key_position[20])
# ひらがな配列をローマ字に変換する関数
def convert_to_romaji(hiragana_string):
  hiragana_to_romaji = {
    "あ": "a", "い": "i", "う": "u", "え": "e", "お": "o",
    "か": "ka", "き": "ki", "く": "ku", "け": "ke", "こ": "ko",
    "さ": "sa", "し": "si", "す": "su", "せ": "se", "そ": "so",
    "た": "ta", "ち": "ti", "つ": "tu", "て": "te", "と": "to",
    "な": "na", "に": "ni", "ぬ": "nu", "ね": "ne", "の": "no",
    "は": "ha", "ひ": "hi", "ふ": "fu", "へ": "he", "ほ": "ho",
    "ま": "ma", "み": "mi", "む": "mu", "め": "me", "も": "mo",
    "や": "ya",             "ゆ": "yu",             "よ": "yo",
    "ら": "ra", "り": "ri", "る": "ru", "れ": "re", "ろ": "ro",
    "わ": "wa",                                     "を": "wo", "ん": "nn",
    "が": "ga", "ぎ": "gi", "ぐ": "gu", "げ": "ge", "ご": "go",
    "ざ": "za", "じ": "ji", "ず": "zu", "ぜ": "ze", "ぞ": "zo",
    "だ": "da", "ぢ": "ji", "づ": "zu", "で": "de", "ど": "do",
    "ば": "ba", "び": "bi", "ぶ": "bu", "べ": "be", "ぼ": "bo",
    "ぱ": "pa", "ぴ": "pi", "ぷ": "pu", "ぺ": "pe", "ぽ": "po",
    "ゔ": "vu",
    "ぁ": "la", "ぃ": "li", "ぅ": "lu", "ぇ":"le", "ぉ":"lo"
  }
  lower_vowel = {"ぁ": "a", "ぃ": "i", "ぅ": "u", "ぇ":"e", "ぉ":"o"}
  y_column_vowel = {"ゃ":"a", "ゅ": "u", "ょ": "o"}

  def two_letter_input(cur_char, next_char,flg):
    if cur_char in ("ふ", "ゔ") and next_char in lower_vowel:
      con = hiragana_to_romaji.get(cur_char,"")[0]
      vowel = lower_vowel.get(next_char,"")
      temp = con + vowel
      flg += 1
    elif next_char in ("ゃ","ゅ","ょ"):
      con = hiragana_to_romaji.get(cur_char,"")[0]
      vowel = y_column_vowel.get(next_char,"")
      temp = con + "y" + vowel
      flg += 1
    else:
      temp=""
      flg=0
    return temp, flg

  def lower_case_tu(cur_char, next_char, next_next_char,flg):
    con = hiragana_to_romaji.get(next_char,"")[0]
    if next_next_char in lower_vowel or next_next_char in y_column_vowel:
      temp, flg = two_letter_input(next_char, next_next_char, flg)
      temp = con + temp
      flg += 1
    else:
      vowel = hiragana_to_romaji.get(next_char,"")[1]
      temp = con *2 + vowel
      flg += 1
    return temp, flg

  romaji_result = ""
  i = 0
  r=-1
  while i < len(hiragana_string):
    if i==r:
      print("@@@@@")
      break
    r= i

    flg = 0
    cur_char = hiragana_string[i]
    if i < len(hiragana_string)-1:
      next_char = hiragana_string[i+1]
    else:
      next_char =""
    if i < len(hiragana_string)-2:
      next_next_char = hiragana_string[i+2]
    else:
      next_next_char =""
    # 分岐スタート
    #小さいっの場合
    if cur_char == "っ":
        # 次の文字が存在する場合、その文字の先頭を重ねる
      if next_char:
        temp, flg = lower_case_tu(cur_char, next_char,next_next_char,flg)
        romaji_result += temp
        flg += 1
    #2-letterの場合
    elif next_char in lower_vowel or next_char in y_column_vowel:
      temp, flg = two_letter_input(cur_char, next_char,flg)
      if not temp:
        con =hiragana_to_romaji.get(cur_char)
        lower = hiragana_to_romaji.get(next_char)
        temp = con+ lower
        flg+=1
      romaji_result += temp
      flg += 1
    #1対1変換
    else:
      romaji_result += hiragana_to_romaji.get(cur_char,"")
      flg += 1
    i += flg
  return romaji_result

def cost(prev_key, key):
  #連続して同じ手を用いるか？ 左手: 0 、右手: 1
  cost_additional = 0
  prev_hand = 0 if prev_key[0] <= 4 else 1
  cur_hand = 0 if key[0] <= 4 else 1
  #全く同じなら追加コストなし
  if prev_hand == cur_hand and not np.all(prev_key == key):
    cost_additional += 10
      #同じ側の同じ指を連続して使うならさらにコスト増加
    if (prev_key[0] in (3,4) and key[0] in (3,4)) or (prev_key[0] in (5,6) and key[0] in (5,6)):
      cost_additional += 6
    if prev_key[0] == key[0]:
      cost_additional += 6
  return key_cost[key[1],key[0]] + cost_additional

#key: 入力したキーの座標
#to be done: 上下動作の非対称性、2個飛ばしが起きた際のコスト増加、右手はEnterを使うのでコストが大きめ
# def calc_avg_cost(input_list, key_position):
#   total_cost = 0
#   prev_key =key_position[alphabet_index.get(input_list[0])]
#   for letter in input_list[1:]:
#     key = key_position[alphabet_index.get(letter)]
#     total_cost += cost(prev_key, key)
#   return total_cost/len(input_list)

def calc_transition_N(text):
  transition_N = np.zeros((26,26))
  prev_index = alphabet_index.get(text[0])
  for letter in text[1:]:
    index = alphabet_index.get(letter)
    transition_N[prev_index, index] += 1
    prev_index = index
  return transition_N

def calc_avg_cost(transition_N, key_position):
  N = np.sum(transition_N)
  cost_matrix = np.zeros((26,26))
  for p in range(cost_matrix.shape[0]):
    prev_key = key_position[p]
    for q in range(cost_matrix.shape[1]):
      key = key_position[q]
      cost_matrix[p,q] = math.pow(cost(prev_key, key), 2) * math.log(transition_N[p,q],7) / N if transition_N[p,q] !=0 else 0.00001
  return cost_matrix.sum()

def calc_cost_matrix(transition_N, key_position):
  N = np.sum(transition_N)
  cost_matrix = np.zeros((26,26))
  for i in range(cost_matrix.shape[0]):
    prev_key = key_position[i]
    for j in range(cost_matrix.shape[1]):
      key = key_position[j]
      cost_matrix[i,j] = math.pow(cost(prev_key, key), 2) * math.log(transition_N[i,j],7) / N if transition_N[i,j] !=0 else -0.01
  return cost_matrix

def left_or_right(key_position):
  #各アルファベットが右か左か
  array_right = []
  array_left  = []
  for i in range(key_position.shape[0]):
    position = key_position[i,:]
    letter = index_alphabet.get(i)
    if position[0] > 4:
      array_right.append(letter)
    else:
      array_left.append(letter)

  return array_left, array_right

def load_txt(txt_file):
  with open(txt_file, 'r', encoding='UTF-8',errors = "replace") as file:

      sentence = file.read()
  #print(sentence)
  api_key = 'c7bb63f0be4f3f6f9aa980e67151d12e06dfc6f81937d2e9a244bc8e9d1f6f53'
  #sentence = '皆さんの誤解を解く目的で解説する。あっちゃーヴぁー'
  url = "https://labs.goo.ne.jp/api/hiragana"
  params = {
      "app_id": api_key,
      "sentence": sentence,
      "output_type": "hiragana"
  }
  try:
    response = requests.post(url, json = params)
    if response.status_code == 200:
      sentence_hiragana = response.json()
    else:
      print(f"error: {response.status_code}, message: {response.text}")
  except requests.exceptions.RequestException as e:
      # リクエスト中にエラーが発生した場合
      print("An error has occurred", str(e))
  converted_txt = sentence_hiragana["converted"]
  #print(converted_txt)
  romaji_txt = convert_to_romaji(converted_txt)
  print(romaji_txt)
  input_list = list(romaji_txt)
  input = np.array(input_list)
  return romaji_txt, input_list

def optimize_for_candidates(target_index, key_position, candidates_letter_list, cur_avg_cost,input_list, transition_N):
  new_key_position = key_position.copy()
  flg = 0
  for candidate in candidates_letter_list:
    temp_key_position = key_position.copy()
    #アルファベットのインデックス
    candidate_index = alphabet_index.get(candidate)
    temp = key_position[target_index[1]]
    temp_key_position[target_index[1]] = key_position[candidate_index]
    temp_key_position[candidate_index] = temp
    temp_cost = calc_avg_cost(transition_N, temp_key_position)
    if temp_cost < cur_avg_cost:
      cur_avg_cost = calc_avg_cost(transition_N, temp_key_position)
      new_key_position = temp_key_position
      flg = 1
      print("improved(⇒", candidate,"):", cur_avg_cost)
  return new_key_position, cur_avg_cost, flg

[6 2]


In [221]:
romaji_txt, input_list =load_txt('sample_text_営業意思決定支援.txt')
#transition_Nはkey_positionに依存しないため、最適化最中に更新する必要ない
transition_N = calc_transition_N(romaji_txt)  #df = pd.DataFrame(transition_N,alphabet_index.keys(), alphabet_index.keys()) display(df)
print(transition_N)

...
[[ 28.  20.   0.  15.  45.   4.  59.  27. 279.  18. 320.   0.  40. 360.
    5.  19.   0. 195. 110. 172.   9.   0.  26.   0.  11.  10.]
 [ 26.   0.   0.   0.   9.   0.   0.   0.  44.   0.   0.   0.   0.   0.
    6.   0.   0.   0.   0.   0.  44.   0.   0.   0.   1.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ 35.   0.   0.   0. 212.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  123.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ 34.   3.   0.   4.  27.   8.  12.  41. 248.  24. 101. 195.  86. 250.
    6.  14.   0. 204. 167. 145.   0.   0.   4.   0.   2.   3.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   5.   0.   0.   0.   0.   0.
    1.   0.   0.   0.   0.   0.  38.   0.   0.   0.   0.   0.]
 [166.   0.   0.   0.  24.   0.   0.   0.  14.   0.   0.   0.   0.   0.
   26.   0.   0.   0.   0.   0.  25.   0.   0.   0.  65.   0.]
 [144.   0.   0.   0.  18.   0.   0.   0.  19.   0.

In [222]:
def create_new_key_position(cur_key_position, cur_avg_cost, transition_N):
  def process_target_letter(target_letter,cur_key_position_sub, cur_avg_cost,df_cost_contribution):
    #変えるアルファベットとそのもとになるアルファベットを保存
    target_index = list(map(lambda x:alphabet_index.get(x), target_letter))
    target_letter_position = cur_key_position_sub[target_index[1],:]
    #2:
    cost_for_each_key = df_cost_contribution.sum().to_frame(name = "Sum")
    candidates = cost_for_each_key.loc[array_left].sort_values('Sum',ascending=True) if target_letter_position[0] > 4 else cost_for_each_key.loc[array_right].sort_values('Sum',ascending = True)
    #3: 候補の中でコストを最小化する置換先を求めて交換したnew_key_positionを求める
    #候補となるkeyのリスト（アルファベット）
    candidates_letter_list = candidates.index.tolist()
    #配列を関数に渡すと
    new_key_position, new_avg_cost, flg = optimize_for_candidates(target_index,cur_key_position_sub.copy(),candidates_letter_list,cur_avg_cost, input_list, transition_N)#関数の引数に配列を入力すると、参照渡しされてしまい、元の配列が変更されてしまう⇒.copy()の利用
    return new_key_position, new_avg_cost, flg


  #df_cost_contributionは使用されていない。必要なのはその列和
  df_cost_contribution = pd.DataFrame(calc_cost_matrix(transition_N.copy(), cur_key_position.copy()),alphabet_index.keys(), alphabet_index.keys())

  array_left, array_right = left_or_right(cur_key_position)

  i = 0
  flg = 0
  while(i < 675):
      #1:コスト寄与率が大きい、上からi+1番目のデータを取得
    top_n = i + 1   # 上位N個を取得
    sorted_values = df_cost_contribution.stack().nlargest(top_n)
    target_letter = sorted_values.tail(1).index.tolist()[0]
    new_key_position, new_avg_cost, flg = process_target_letter(target_letter, cur_key_position.copy(),cur_avg_cost, df_cost_contribution.copy())
    if flg == 1:
      break
    else:
      i += 1
  return new_key_position, new_avg_cost, flg


In [225]:
cur_avg_cost = calc_avg_cost(transition_N, key_position)
cur_key_position = key_position.copy()
print('start:', cur_avg_cost)
k = 0
while(k < 100000):
  new_key_position, new_avg_cost, flg = create_new_key_position(cur_key_position.copy(), cur_avg_cost, transition_N)
  if flg == 0:
    print('converged')
    print(new_key_position)
    print(cur_avg_cost)
    break
  else:
    k += 1
    cur_key_position = new_key_position.copy()
    cur_avg_cost = new_avg_cost
    print(f'trial {k} cost: {cur_avg_cost}')
print(cur_avg_cost)
#コスト寄与率を計算

#現在のキー配置とその遷移N数行列より、select target and then select a letter to be replaced located on the other side of the keyboard
#simulates the cost evaluation for the selected replacement. the replacement measured the lowest cost would be new_key_position
#if nothing should be replaced (flg = 0), do the same procedure for the biggest six contributors
#if i hits the maximum target, exit. And the produced key_position will be the best arrangement
  # display(cost_for_each_key)
  # display(candidates)
  # display(df_cost_contribution)
  #⇒uとnの位置を変更したい。
  #まず、どのキーが次に続くキーがどちら側に来やすいかを求めて初期の配列を構成すべきか？
  #目指す方向としては、同じ側で連続して打つ頻度を減らすこと（こちらは上記の初期化でうまくいくかもしれない、頻度をそのまま使うことはできないので、割合で計算する必要がある。）
  #打つ頻度の大きなキーはできるだけホームポジションに位置させること（初期化した中で頻度の大きいものをホームポジションに配置する。さらに、同じ手の中で連続）

  #1:コスト行列内から最も値の大きな(選択方法は要検討)要素targetを選ぶ
  #2:コスト行列のcolumn和を取りその中で最も値(合計寄与率)が小さな文字replacedを取得:N=0の値は-0.001ぐらいが妥当か,この際、targetのrowの文字と異なる側から選択する(逆サイドの文字全てについてシミュレーションを行い、コスト関数を最も減少させる組み合わせを求めてもいいかも)
  #targetのcolumnの文字とreplacedの位置を交換する(key_position)の更新
  #新しくコスト行列を計算
  #εGREEDYを使うのもありかも、(確率εでランダムに位置変更)次第にεの値を縮小していくー同じキーの交換が続いてしまうかも

start: 6.600127220724161
improved(⇒ c ): 6.318297764297242
trial 1 cost: 6.318297764297242
improved(⇒ v ): 6.310036588467698
improved(⇒ f ): 6.065170445277414
improved(⇒ d ): 5.858649632898202
trial 2 cost: 5.858649632898202
improved(⇒ v ): 5.559722514590787
improved(⇒ f ): 5.483517444826809
trial 3 cost: 5.483517444826809
improved(⇒ v ): 5.204281464640198
trial 4 cost: 5.204281464640198
improved(⇒ g ): 5.1689530402051584
improved(⇒ n ): 5.092814775241056
improved(⇒ e ): 5.006495584364904
improved(⇒ a ): 4.752134949545171
trial 5 cost: 4.752134949545171
improved(⇒ n ): 4.710043099630962
trial 6 cost: 4.710043099630962
improved(⇒ l ): 4.364856346521705
improved(⇒ k ): 4.235678881904182
trial 7 cost: 4.235678881904182
improved(⇒ o ): 4.21846154999148
trial 8 cost: 4.21846154999148
improved(⇒ h ): 4.150978632921216
trial 9 cost: 4.150978632921216
improved(⇒ x ): 4.074426353680916
improved(⇒ n ): 4.050778743541472
trial 10 cost: 4.050778743541472
improved(⇒ q ): 3.9189474181177295
improved

In [235]:
draw = np.full((3, 10), "none")

for i in range(new_key_position.shape[0]):
  draw[2-new_key_position[i,1], new_key_position[i,0]] = index_alphabet.get(i)
display(draw)

array([['l', 'w', 'k', 'r', 't', 'x', 'u', 'i', 'o', 'v'],
       ['m', 's', 'd', 'y', 'g', 'n', 'j', 'e', 'q', 'none'],
       ['z', 'f', 'h', 'p', 'b', 'c', 'a', 'none', 'none', 'none']],
      dtype='<U4')